# VROOM testing without ORS

In [1]:
import requests
import folium
import json
import random
from pprint import pprint

# DATI json posizione jobs (Bidoni)
pointsA = json.load(open('./data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('./data/mezzocorona.geojson'))['features'] #6 points

#shipments = json.load(open('./data/shipments.json'))

center = [46.2147, 11.1065]
zoom = 15

# POSIZIONI PARTENZA DEI VEICOLI (Furgoni)
point_start = [46.2170, 11.1066]

# POSIZIONE ARRIVO DEI VEICOLI
point_end = [point_start[0], point_start[1]]

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]
#ELENCO BIDONI 4 Mezzolombardo e 6 Mezzocorona TOTALE 10 con CONTENUTO CASUALE
job_locs = ccA + ccB


# BUILD MAP from DATA

map1 = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

vehicles = []

#vehicle capacity
capacity = 5

vehicles.append({
    'id': 1,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': list(reversed(point_start)), # posizione di partenza veicoli
    'end': list(reversed(point_start))
})
mark = folium.Marker(point_start, tooltip=folium.Tooltip(capacity,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color='blue'))
mark.add_to(map1)

#vehicle capacity
capacity = 5
point_startB = [point_start[0] + 0.0015, point_start[1] - 0.0015]

vehicles.append({
    'id': 1,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': list(reversed(point_startB)), # posizione di partenza veicoli
    'end': list(reversed(point_startB))
})
mark = folium.Marker(point_start, tooltip=folium.Tooltip(capacity,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color='blue'))
mark.add_to(map1)

jobs = []
for idx, cc in enumerate(job_locs):
    loc = list(reversed(cc))
    
    pickup = random.randrange(1, 3) #random value of bin filling
    
    mark = folium.Marker(loc, tooltip=folium.Tooltip(pickup, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(map1)

    jobs.append({
        'id': idx,
        'location': cc,
        'pickup': [pickup] # Quantita' contenuta nel Bidone
        #priority = [1-100] potrebbe essere utile anche priority
        #ma non e' supportato, bug che verra presto risolto https://github.com/GIScience/openrouteservice-py/issues/39
    })

map1

In [2]:
# PRINT VROOM REQUEST

print(json.dumps({'vehicles': vehicles, 'jobs': jobs }))

{"vehicles": [{"id": 1, "capacity": [5], "profile": "driving-car", "start": [11.1066, 46.217], "end": [11.1066, 46.217]}, {"id": 1, "capacity": [5], "profile": "driving-car", "start": [11.1051, 46.2185], "end": [11.1051, 46.2185]}], "jobs": [{"id": 0, "location": [11.092028617858887, 46.2173172744161], "pickup": [1]}, {"id": 1, "location": [11.08793020248413, 46.2161294935962], "pickup": [1]}, {"id": 2, "location": [11.091685295104979, 46.21057627744751], "pickup": [2]}, {"id": 3, "location": [11.096470355987549, 46.21344204466888], "pickup": [2]}, {"id": 4, "location": [11.112627983093262, 46.21758452155989], "pickup": [2]}, {"id": 5, "location": [11.118206977844238, 46.214911991592125], "pickup": [2]}, {"id": 6, "location": [11.123356819152832, 46.21419929496722], "pickup": [1]}, {"id": 7, "location": [11.127262115478516, 46.211972058400264], "pickup": [2]}, {"id": 8, "location": [11.122541427612305, 46.20986352455638], "pickup": [1]}, {"id": 9, "location": [11.114301681518555, 46.20

In [5]:
vroom_url = 'http://vroom:3000/optimization/'

params = {
    'vehicles': vehicles,
    'jobs': jobs,
    'geometry': True
}
resp = requests.post(vroom_url, json = params)

print(resp.status_code)

solution = resp.json()

solution



500


{'code': 3,
 'error': 'Request parameters exceed the server configuration limits. Only a total of 100 routes are allowed.'}

In [ ]:
solutionA = solution['routes'][0]

poly = folium.PolyLine(
    locations = [list(reversed(coords)) for coords in ors.convert.decode_polyline(solutionA['geometry'])['coordinates']],
    color = 'blue',
    tooltip = folium.Tooltip('Distance: '+str(solutionA['distance']/1000)+'km', permanent=True)
)

poly.add_to(map1)

map1

In [ ]:
solution